In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.preprocessing import OneHotEncoder


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import gc
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')

In [ ]:
train_oof = np.zeros((train.shape[0],9))
test_preds = 0
train_oof.shape

In [ ]:
columns = test.columns
target = train['target'].values
train = train[columns].values
test = test[columns].values

In [ ]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(train)):
    print("Fitting fold", jj+1)
    train_features = train[train_index]
    train_target = target[train_index]
    
    val_features = train[val_index]
    val_target = target[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=600, learning_rate=0.01, max_depth=24, 
                                           l2_regularization = 0.012, max_bins=255, 
                                           early_stopping=False, validation_fraction=None)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    #print("Fold AUC:", roc_auc_score(val_target, val_pred[:,1]))
    test_preds += model.predict_proba(test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

In [ ]:
oho = OneHotEncoder()
log_loss(oho.fit_transform(target.reshape(-1,1)).toarray(), train_oof)

In [ ]:
sample_submission[sample_submission.columns[1:]] = test_preds
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv', index=False)